In [13]:
# pip install requests BeautifulSoup csv json time random

import csv
import json

import requests
from bs4 import BeautifulSoup

import time
import random

# 1. HEADER
Primeramente vamos a agregar un header, este está en las opciones de desarrolladores en la página web


In [14]:
# Esto simula un scraping con la configuración del navegador
header = {
    "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Mobile Safari/537.36"
}

In [15]:
base_url = "http://books.toscrape.com/catalogue/category/books_1/page-{}.html"

Recorremos todas la paginas para guardar los productos

In [16]:
product_list = [] #  Lista de productos

# recorremos las 4 pestañas
for page in range(1, 10):
    url = base_url.format(page) #  formateamos la base url para añadir las paginas
    response = requests.get(url)  # obtenemos la información de la pagina
    soup = BeautifulSoup(response.text, "html.parser")  # parseamos el html obtenido
    products = soup.select("article.product_pod") #  Seleccionamos la lista de productos en su elemento



    # procesamos cada producto
    for product in products:
        title = product.find("h3").find("a")["title"]
        price = product.find("p", class_="price_color").get_text()
        image_rel = product.find("div", class_="image_container").find("img")["src"]
        img_url = "http://books.toscrape.com/" + image_rel

        # prearar el json
        product_list.append({
            "title": title,
            "price": price,
            "img_url": img_url
        })

    # Se deja el sleep al mismo nivel del primer ciclo

    # Espera breve entre paginas, así se simula navegación Real
    sleep_time = random.uniform(1, 3)
    # como buscamos simular a un humano, vamos a ingresar request en tiempos diferentes
    time.sleep(sleep_time)
    print(f"Pagina {page} procesada con una pausa de {sleep_time}")

Pagina 1 procesada con una pausa de 2.503753124219253
Pagina 2 procesada con una pausa de 2.5518467577706057
Pagina 3 procesada con una pausa de 1.1502679373255356
Pagina 4 procesada con una pausa de 2.278601528111455
Pagina 5 procesada con una pausa de 2.8112147586246623
Pagina 6 procesada con una pausa de 1.3710507711570432
Pagina 7 procesada con una pausa de 1.5682795752838172
Pagina 8 procesada con una pausa de 1.4666380706656224
Pagina 9 procesada con una pausa de 2.955837548476963


In [17]:
with open("resultados/productos_mult_page.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=["title", "price", "img_url"])
    writer.writeheader()
    writer.writerows(product_list)

print(f"Scraping multipágina completado: {len(product_list)} productos guardados en el documento.")


Scraping multipágina completado: 180 productos guardados en el documento.


# 2. Manejo de excepciones y cambios de url

In [31]:
product_list = []  # Lista de productos

# recorremos las 4 pestañas
for page in range(1, 56):
    # formateamos la base url para añadir las paginas
    url = base_url.format(page)

    # añadiremos persistencia ante errores
    try:    
        response = requests.get(url)  # obtenemos la información de la pagina
        # IMPORTANTE
        response.raise_for_status() #  Este lanza los codigos de status
        # parseamos el html obtenido
        soup = BeautifulSoup(response.text, "html.parser")
        # Seleccionamos la lista de productos en su elemento
        products = soup.select("article.product_pod")

    except requests.RequestException as e:
        # muestra el error
        print(f"Error en la página {page}: {e}")
        continue # continua a la siguiente iteración

    # procesamos cada producto
    for product in products:
        try:
            title = product.find("h3").find("a")["title"]
            price = product.find("p", class_="price_color").get_text()
            image_rel = product.find(
                "div", class_="image_container").find("img")["src"]
            img_url = "http://books.toscrape.com/" + image_rel

            # prearar el json
            product_list.append({
                "title": title,
                "price": price,
                "img_url": img_url
            })

        # Manejo de posibles errores y excepciones   
        except Exception as ex:
            print("Error extrayendo datos de un producto: ", ex)

    # Espera breve entre paginas, así se simula navegación Real
    sleep_time = random.uniform(1, 3)
    # como buscamos simular a un humano, vamos a ingresar request en tiempos diferentes
    time.sleep(sleep_time)
    print(f"Pagina {page} procesada con una pausa de {sleep_time}")

Pagina 1 procesada con una pausa de 2.1547217941300656
Pagina 2 procesada con una pausa de 1.7680068064810854
Pagina 3 procesada con una pausa de 1.539382266943058
Pagina 4 procesada con una pausa de 2.8295036058300904
Pagina 5 procesada con una pausa de 1.7123562769524723
Pagina 6 procesada con una pausa de 2.866728683412272
Pagina 7 procesada con una pausa de 1.702026726439344
Pagina 8 procesada con una pausa de 1.7298865864344348
Pagina 9 procesada con una pausa de 1.2068959323879662
Pagina 10 procesada con una pausa de 2.0359335823303173
Pagina 11 procesada con una pausa de 1.064479685755086
Pagina 12 procesada con una pausa de 2.6119104154987687
Pagina 13 procesada con una pausa de 1.895831850389478
Pagina 14 procesada con una pausa de 1.4092860337065451
Pagina 15 procesada con una pausa de 1.2097595175063742
Pagina 16 procesada con una pausa de 1.7315728057178015
Pagina 17 procesada con una pausa de 2.4187242490857965
Pagina 18 procesada con una pausa de 2.4758825200287253
Pagina

In [32]:
# guardar resultados en CSV
path_csv = "resultados/productos_eticos.csv"
# newline -> agregar algo al principio
with open(path_csv, "w", newline="", encoding="utf-8") as f:
    try:
        writer = csv.DictWriter(f, fieldnames=["title", "price", "img_url"])
        writer.writeheader()  # crear cabercera con los fieldnames
        writer.writerows(product_list)
    except Exception as e:
        print(f"Error en la creación de archivo csv: {e}")
print(f"Extracción completa: {len(product_list)} productos guardados")

Extracción completa: 1000 productos guardados


# 3. Procesar datos en JSON

In [33]:
# pip install pandas openpyxl
import pandas as pd
import openpyxl

In [34]:
# Convertir en excel los resultados/
df = pd.DataFrame(product_list)


# guardar como archivo excel
df.to_excel("resultados/productos_eticos.xlsx", index=False)

print("Scraping ético completado: {len(product_list)} productos guardados en formato excel en productos_eticots.xlsx")


Scraping ético completado: {len(product_list)} productos guardados en formato excel en productos_eticots.xlsx


# 4. Guardar en un Google Form
Es una forma sin api y sin autenticación

https://docs.google.com/forms/d/e/1FAIpQLSckn8kqx6ZVXSkxUxkTvlZjd1ESm1XSnkyiurWO35qX-6Wzbg/viewform?usp=pp_url&entry.393050659=pinocho&entry.1446703210=15000&entry.778820532=www.pinocho.com

In [35]:

google_form_url = "https://docs.google.com/forms/d/1Vf0AkZ54goAiRyLDp22cyQBlo2BG2doP_KfJBeUOJeg/prefill"
google_form_id = "1FAIpQLSckn8kqx6ZVXSkxUxkTvlZjd1ESm1XSnkyiurWO35qX-6Wzbg"
google_form_link = "https:docs.google.com/forms/d/e/{}/viewform"

In [36]:
# url para post
url = "https://docs.google.com/forms/d/e/{}/formResponse"
# headers
headers = {
    "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Mobile Safari/537.36",
    "Referer": google_form_link.format(google_form_id)
}

# Recorrer y enviar cada producto

for i, product in enumerate(product_list[0:], start=1):

    payload = {
        "entry.393050659" : product["title"], #  Enviamos el titulo
        "entry.1446703210" : product["price"], #  Enviamos el precio
        "entry.778820532" : product["img_url"] #  Enviamos el img
    }

    # enviar al formulario la información del producto
    response = requests.post(url.format(google_form_id), data=payload, headers=headers)

    if response.status_code == 200:
        print(f"Producto {i} enviado: {product['title']}")
    else:
        print(f"Error al enviar producto {i} -  Código: {response.status_code}")

    time.sleep(sleep_time)

Producto 1 enviado: A Light in the Attic
Producto 2 enviado: Tipping the Velvet
Producto 3 enviado: Soumission
Producto 4 enviado: Sharp Objects
Producto 5 enviado: Sapiens: A Brief History of Humankind
Producto 6 enviado: The Requiem Red
Producto 7 enviado: The Dirty Little Secrets of Getting Your Dream Job
Producto 8 enviado: The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull
Producto 9 enviado: The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics
Producto 10 enviado: The Black Maria
Producto 11 enviado: Starving Hearts (Triangular Trade Trilogy, #1)
Producto 12 enviado: Shakespeare's Sonnets
Producto 13 enviado: Set Me Free
Producto 14 enviado: Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)
Producto 15 enviado: Rip it Up and Start Again
Producto 16 enviado: Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991
Producto 17 enviado: Olio
Producto 18 enviado: Mesaerion: 